In [1]:
import numpy as np
from sympy import *
import sympy as sp
init_printing()
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
init_vprinting()

In [146]:
a, b, a3, a4, b3, b4, c, rdw = symbols('a b a_3 a_4 b_3 b_4 c r_dw', type='real')
m, g, I = symbols('m g I', type='real') # g is positive 9.8; I is around center of mass
t3, t4, rc = symbols('theta_3 theta_4 r_c', type='real')
v, w = dynamicsymbols('v omega')
t = symbols('t')

urc, urd, ut, eps = symbols('mu_rc mu_rd mu_tau epsilon')

# COM And Weight Force
Ox, Oy, Oz = symbols('O_x O_y O_z')
O = Matrix([Ox, Oy, Oz])

def rp_matrix(r, p):
    Rx = Matrix([[1, 0, 0], [0, cos(r), -sin(r)], [0, sin(r), cos(r)]])
    Ry = Matrix([[cos(p), 0, sin(p)], [0, 1, 0], [-sin(p), 0, cos(p)]])
    return Ry*Rx

roll, pitch = symbols('theta_roll, theta_pitch')
Fg = rp_matrix(roll, pitch).T * (Matrix([0, 0, -m*g]))

#Standard vectors:
z_vec = Matrix([0,0,0])
ivec = Matrix([1,0,0])
jvec = Matrix([0,1,0])
kvec = Matrix([0,0,1])

In [148]:
# For keeping all variable definitions
from sympy.printing.cxxcode import cxxcode
def make_var_list():
    strings = []
    names = []

    def vardef(name, xpr):
        s = cxxcode(xpr, name)
        if name in names:
            ind = names.index(name)
            strings[ind] = s
        else:
            strings.append(s)
            names.append(name)
        return s
    
    def add_manual(name, string):
        if name in names:
            ind = names.index(name)
            strings[ind] = string
        else:
            strings.append(string)
            names.append(name)
        return string
    
    def get_all():
        return strings
    
    return vardef, add_manual, get_all
vardef, add_manual, get_all = make_var_list()

# Normal Forces

In [149]:
zo, theta, phi = symbols('z_o theta phi', type='real')
k, n = symbols('k n', type='real')
k1, k2 = k, n*k

#z1 = zo + theta*x1 - phi*y1
z1_xpr = zo + theta*c
z2_xpr = zo + theta*(-c)

#z3 = xo + theta*(x3) - phi*y3
z3_xpr = zo + theta*(a3) - phi*(b3)
z4_xpr = zo + theta*(-a4) - phi*(b4)

#N_i = z_i * k_i
N1 = -k1*z1_xpr
N2 = -k1*z2_xpr
N3 = -k2*z3_xpr
N4 = -k2*z4_xpr

N1 = N1.expand().collect([zo, theta, phi])
N2 = N2.expand().collect([zo, theta, phi])
N3 = N3.expand().collect([zo, theta, phi])
N4 = N4.expand().collect([zo, theta, phi])

In [150]:
W_z = -Fg.dot(kvec)
Tnet_x = -O.cross(Fg).dot(ivec) - v.diff(t)*m*Oz**2
Tnet_y =  -O.cross(Fg).dot(jvec) - m*w.diff(t)*Oz

In [151]:
Line_1 = N1 + N2 + N3 + N4 # Equals Fw dot k_hat
Line_1 = Line_1.expand().collect([zo, theta, phi])

Eq_1 = Eq(Line_1, W_z)
Eq_1

In [152]:
Line_2 = -N3*b3 - N4*b4 # Equals -(O x Fg) dot i_hat + vdot * m *  O_z**2
Line_2 = Line_2.expand().collect([zo, theta, phi])

Eq_2 = Eq(Line_2, Tnet_x)
Eq_2

In [153]:
Line_3 = N1*c - N2*c + N3*a3 - N4*a4
Line_3 = Line_3.expand().collect([zo, theta, phi])

Eq_3 = Eq(Line_3, Tnet_y)
Eq_3

In [154]:
solns = solve([Eq_1, Eq_2, Eq_3], [zo, theta, phi])

phi_expr = solns[phi]
theta_expr = solns[theta]
zo_expr = solns[zo]

q = Matrix([[zo_expr], [theta_expr], [phi_expr]])
q

⎡  ⎛  ⎛                   2   ⎛  2     2⎞ ⎛  2       2        2⎞⎞             
⎢m⋅⎝g⋅⎝- n⋅(a₃⋅b₃ - a₄⋅b₄)  + ⎝b₃  + b₄ ⎠⋅⎝a₃ ⋅n + a₄ ⋅n + 2⋅c ⎠⎠⋅cos(θ_pitch)
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                  ⎛      ⎛          ⎛  2     2⎞                            ⎞ 
⎢                m⋅⎝- g⋅n⋅⎝(a₃ - a₄)⋅⎝b₃  + b₄ ⎠ - (b₃ + b₄)⋅(a₃⋅b₃ - a₄⋅b₄)⎠⋅
⎢                ─────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢    ⎛      ⎛                                       

In [155]:
add_manual('a_3', "a_3 = a + r_c * std::cos(theta_3)")
add_manual('a_4', "a_4 = a + r_c * std::cos(theta_4)")
add_manual('b_3', "b_3 = b - r_c * std::sin(theta_3)") # MINUS rc*sin(theta_3) because b is a in the negative direction
add_manual('b_4', "b_4 = b - r_c * std::sin(theta_4)") # MINUS because b is in negative direction

vardef("zo", zo_expr)
vardef("theta", theta_expr)
vardef("phi", phi_expr)

vardef("N_1", N1)
vardef("N_2", N2)
vardef("N_3", N3)
vardef("N_4", N4)

'N_4 = a_4*k*n*theta + b_4*k*n*phi - k*n*z_o;'

# Castors

In [40]:
#Remake normal force scalars as symbols
N1, N2, N3, N4 = symbols('N_1 N_2 N_3 N_4', type='real')

rc3 = Matrix([rc*cos(t3), rc*sin(t3), 0])
rc4 = Matrix([rc*cos(t4), rc*sin(t4), 0])
v3 = Matrix([b*w, v - a*w, 0])
v4 = Matrix([b*w, v + a*w, 0])

In [121]:
# Rolling Friction Terms
s_f3_xpr = Piecewise((0, v3.dot(v3) < eps), ((rc3.dot(v3) / (rc * sqrt(v3.dot(v3)))).simplify(), True))
s_f4_xpr = Piecewise((0, v4.dot(v4) < eps), ((rc4.dot(v4) / (rc * sqrt(v4.dot(v4)))).simplify(), True))
s_f3, s_f4 = symbols('s_f3 s_f4')

Rf3_xpr = Piecewise((-rc3/rc * urc*N3, s_f3 > eps), (rc3/rc * urc*N3, s_f3 < -eps), (z_vec, True))
Rf4_xpr = Piecewise((-rc4/rc * urc*N4, s_f4 > eps), (rc4/rc * urc*N4, s_f4 < -eps), (z_vec, True))

Rf3x_xpr = Piecewise(((-rc3/rc * urc*N3).dot(ivec), s_f3 > eps), ((rc3/rc * urc*N3).dot(ivec), s_f3 < -eps), (0, True))
Rf3y_xpr = Piecewise(((-rc3/rc * urc*N3).dot(jvec), s_f3 > eps), ((rc3/rc * urc*N3).dot(jvec), s_f3 < -eps), (0, True))
Rf4x_xpr = Piecewise(((-rc4/rc * urc*N4).dot(ivec), s_f4 > eps), ((rc4/rc * urc*N4).dot(ivec), s_f4 < -eps), (0, True))
Rf4y_xpr = Piecewise(((-rc4/rc * urc*N4).dot(jvec), s_f4 > eps), ((rc4/rc * urc*N4).dot(jvec), s_f4 < -eps), (0, True))

In [119]:
# Friction Torque Terms
s_t3_xpr = Piecewise((0, v3.dot(v3) < eps), ((rc3.cross(v3)/(rc*sqrt(v3.dot(v3)))).dot(kvec), True))
s_t4_xpr = Piecewise((0, v3.dot(v3) < eps), ((rc4.cross(v4)/(rc*sqrt(v4.dot(v4)))).dot(kvec), True))
s_t3, s_t4 = symbols('s_t3 s_t4')
#Rt3_xpr = Piecewise((-ut*N3 * (rc3.cross(kvec)/rc), s_t3 < -eps), (ut*N3 * (rc3.cross(kvec)/rc), s_t3 > eps), (z_vec, True))
#Rt4_xpr = Piecewise((-ut*N4 * (rc4.cross(kvec)/rc), s_t4 < -eps), (ut*N4 * (rc4.cross(kvec)/rc), s_t4 > eps), (z_vec, True))

Rt3x_xpr = Piecewise(((-ut*N3 * (rc3.cross(kvec)/rc)).dot(ivec), s_t3 < -eps), ((ut*N3 * (rc3.cross(kvec)/rc)).dot(ivec), s_t3 > eps), (0, True))
Rt3y_xpr = Piecewise(((-ut*N4 * (rc4.cross(kvec)/rc)).dot(jvec), s_t4 < -eps), ((ut*N4 * (rc4.cross(kvec)/rc)).dot(jvec), s_t4 > eps), (0, True))
Rt4x_xpr = Piecewise(((-ut*N4 * (rc4.cross(kvec)/rc)).dot(ivec), s_t4 < -eps), ((ut*N4 * (rc4.cross(kvec)/rc)).dot(ivec), s_t4 > eps), (0, True))
Rt4y_xpr = Piecewise(((-ut*N4 * (rc4.cross(kvec)/rc)).dot(jvec), s_t4 < -eps), ((ut*N4 * (rc4.cross(kvec)/rc)).dot(jvec), s_t4 > eps), (0, True))

In [158]:
vardef("s_f3", s_f3_xpr)
vardef("s_t3", s_t3_xpr)

vardef("s_f4", s_f4_xpr)
vardef("s_t4", s_t4_xpr)

vardef("R_3x", Rf3x_xpr + Rt3x_xpr)
vardef("R_3y", Rf3y_xpr + Rt3y_xpr)
vardef("R_4x", Rf4x_xpr + Rt4x_xpr)
vardef("R_4y", Rf4y_xpr + Rt4y_xpr)

'R_4y = ((s_f4 > epsilon) ? (\n   -N_4*mu_rc*sin(theta_4)\n)\n: ((s_f4 < -epsilon) ? (\n   N_4*mu_rc*sin(theta_4)\n)\n: (\n   0\n))) + ((s_t4 < -epsilon) ? (\n   N_4*mu_tau*cos(theta_4)\n)\n: ((s_t4 > epsilon) ? (\n   -N_4*mu_tau*cos(theta_4)\n)\n: (\n   0\n)));'

# Dynamics

In [44]:
#p1, p2, p3, p4, O = symbols('p_1 p_2 p_3 p_4 O')
#Fw1, Fw2, Fs1, Fs2, Fn1, Fn2, Fn3, Fn4, Fg = symbols('F_w1 F_w2 F_s1 F_s2 F_N1 F_N2 F_N3 F_N4 F_g')

R3x, R3y, R4x, R4y = symbols('R_3x R_3y R_4x R_4y')

p1 = Matrix([-c, 0, 0])
p2 = Matrix([c, 0, 0])
p3 = Matrix([-a, -b, 0])
p4 = Matrix([a, -b, 0]) 

ap1, ap2, bt1, bt2 = symbols('alpha_1 alpha_2 beta_1 beta_2')

Fw1 = Matrix([0, ap1, 0])
Fw2 = Matrix([0, ap2, 0])
Fs1 = Matrix([bt1, 0, 0])
Fs2 = Matrix([bt2, 0, 0])
Fn1 = Matrix([0, 0, N1])
Fn2 = Matrix([0, 0, N2])
Fn3 = Matrix([0, 0, N3])
Fn4 = Matrix([0, 0, N4])
R3 = Matrix([R3x, R3y, 0])
R4 = Matrix([R4x, R4y, 0])

In [45]:
# Sum of Forces
F_LHS = Fw1 + Fw2 + Fs1 + Fs2 + Fn1 + Fn2 + Fn3 + Fn4 + Fg + R3 + R4
F_RHS = Matrix([w.diff(t)*Oy**2*m, v.diff(t)*m, 0])
Eq(F_LHS, F_RHS)

                                                        ⎡   2     ⎤
⎡      R₃ₓ + R₄ₓ + β₁ + β₂ + g⋅m⋅sin(θ_pitch)       ⎤   ⎢O_y ⋅m⋅ω̇⎥
⎢                                                   ⎥   ⎢         ⎥
⎢R_3y + R_4y + α₁ + α₂ - g⋅m⋅sin(θᵣₒₗₗ)⋅cos(θ_pitch)⎥ = ⎢  m⋅v̇   ⎥
⎢                                                   ⎥   ⎢         ⎥
⎣  N₁ + N₂ + N₃ + N₄ - g⋅m⋅cos(θ_pitch)⋅cos(θᵣₒₗₗ)  ⎦   ⎣    0    ⎦

In [46]:
M_LHS = p1.cross(Fw1) + p2.cross(Fw2) + p1.cross(Fs1) + p2.cross(Fs2) + O.cross(Fg) + p1.cross(Fn1) + p2.cross(Fn2) + p3.cross(Fn3) + p4.cross(Fn4) + p3.cross(R3) + p4.cross(R4)
M_RHS = Matrix([v.diff(t)*m*Oz**2, 0, I*w.diff(t)])
Eq(M_LHS, M_RHS)

                                                                              
⎡           -N₃⋅b - N₄⋅b - O_y⋅g⋅m⋅cos(θ_pitch)⋅cos(θᵣₒₗₗ) + O_z⋅g⋅m⋅sin(θᵣₒₗₗ
⎢                                                                             
⎢          N₁⋅c - N₂⋅c + N₃⋅a - N₄⋅a + Oₓ⋅g⋅m⋅cos(θ_pitch)⋅cos(θᵣₒₗₗ) + O_z⋅g⋅
⎢                                                                             
⎣-Oₓ⋅g⋅m⋅sin(θᵣₒₗₗ)⋅cos(θ_pitch) - O_y⋅g⋅m⋅sin(θ_pitch) + R₃ₓ⋅b - R_3y⋅a + R₄ₓ

                             ⎡   2     ⎤
)⋅cos(θ_pitch)           ⎤   ⎢O_z ⋅m⋅v̇⎥
                         ⎥   ⎢         ⎥
m⋅sin(θ_pitch)           ⎥ = ⎢    0    ⎥
                         ⎥   ⎢         ⎥
⋅b + R_4y⋅a - α₁⋅c + α₂⋅c⎦   ⎣  I⋅ω̇   ⎦

In [47]:
eq1 = Eq(F_LHS[1], F_RHS[1]) #Pick equation of Fy
eq2 = Eq(M_LHS[2], M_RHS[2]) # Pick equation of Mz
eq1, eq2

In [48]:
ap_dict = solve([eq1, eq2], [ap1, ap2])
ap1_xpr = ap_dict[ap1]
ap2_xpr = ap_dict[ap2]

In [159]:
vardef("alpha_1", ap1_xpr)
vardef("alpha_2", ap2_xpr)

'// Not supported in C++:\n// Derivative\n// Derivative\nalpha_2 = (1.0L/2.0L)*(I*Derivative(omega(t), t) + O_x*g*m*sin(theta_roll)*cos(theta_pitch) + O_y*g*m*sin(theta_pitch) - R_3x*b + R_3y*a - R_4x*b - R_4y*a - c*(R_3y + R_4y - g*m*sin(theta_roll)*cos(theta_pitch) - m*Derivative(v(t), t)))/c;'

# Drive Wheels

In [50]:
v1 = v - w*c
v2 = v + w*c
Rf_1_xpr = Piecewise((-urd * N1, v1 > eps), (urd*N1, v1 < -eps), (0, True))
Rf_2_xpr = Piecewise((-urd * N2, v2 > eps), (urd*N1, v2 < -eps), (0, True))

Torque_1 = (ap1_xpr - Rf_1_xpr) * rdw
Torque_2 = (ap2_xpr - Rf_2_xpr) * rdw

In [160]:
vardef("torque_1", Torque_1)
vardef("torque_2", Torque_2)

'// Not supported in C++:\n// Derivative\n// Derivative\n// omega\n// v\ntorque_2 = r_dw*(-((c*omega(t) + v(t) > epsilon) ? (\n   -N_2*mu_rd\n)\n: ((c*omega(t) + v(t) < -epsilon) ? (\n   N_1*mu_rd\n)\n: (\n   0\n))) + (1.0L/2.0L)*(I*Derivative(omega(t), t) + O_x*g*m*sin(theta_roll)*cos(theta_pitch) + O_y*g*m*sin(theta_pitch) - R_3x*b + R_3y*a - R_4x*b - R_4y*a - c*(R_3y + R_4y - g*m*sin(theta_roll)*cos(theta_pitch) - m*Derivative(v(t), t)))/c);'

# Translating Sympy to Code

In [142]:
def translate_str(s):
    s3 = s.replace("Derivative(omega(t), t)", "omega_dot")
    s4 = s3.replace("Derivative(v(t), t)", "v_dot")
    s5 = s4.replace("(t)", "")
    s6 = s5.replace("// Not supported in C++:\n", "").replace("// Derivative\n", "")
    s7 = s6.replace("// omega\n", "").replace("// v\n", "")
    return s7

In [161]:
filestr = "// User must define omega and v\n"
for s in get_all():
    filestr += translate_str(s) + "\n" * 2

print(filestr)


// User must define omega and v
a_3 = a + r_c * std::cos(theta_3)

a_4 = a + r_c * std::cos(theta_4)

b_3 = b - r_c * std::sin(theta_3)

b_4 = b - r_c * std::sin(theta_4)

zo = m*(g*(-n*std::pow(a_3*b_3 - a_4*b_4, 2) + (std::pow(b_3, 2) + std::pow(b_4, 2))*(std::pow(a_3, 2)*n + std::pow(a_4, 2)*n + 2*std::pow(c, 2)))*cos(theta_pitch)*cos(theta_roll) + n*((a_3 - a_4)*(std::pow(b_3, 2) + std::pow(b_4, 2)) - (b_3 + b_4)*(a_3*b_3 - a_4*b_4))*(O_x*g*cos(theta_pitch)*cos(theta_roll) + O_z*g*sin(theta_pitch) + O_z*omega_dot) + (n*(a_3 - a_4)*(a_3*b_3 - a_4*b_4) - (b_3 + b_4)*(std::pow(a_3, 2)*n + std::pow(a_4, 2)*n + 2*std::pow(c, 2)))*(-O_y*g*cos(theta_pitch)*cos(theta_roll) + std::pow(O_z, 2)*v_dot + O_z*g*sin(theta_roll)*cos(theta_pitch)))/(k*(std::pow(n, 2)*std::pow(a_3 - a_4, 2)*(std::pow(b_3, 2) + std::pow(b_4, 2)) - 2*std::pow(n, 2)*(a_3 - a_4)*(b_3 + b_4)*(a_3*b_3 - a_4*b_4) + n*std::pow(b_3 + b_4, 2)*(std::pow(a_3, 2)*n + std::pow(a_4, 2)*n + 2*std::pow(c, 2)) + 2*n*(n + 1)*std::pow(

In [34]:
funcstr = """import numpy as np
from numpy import cos, sin, sqrt, array

a = 0.2
b = 0.4
c = 0.2
r_c = 0.05
r_dw = 0.1
m = 230
n = 100
mu_rc = 0.01
mu_rd = 0.08
mu_tau = 0.02

O_x, O_y, O_z = 0, -0.1, 0.25

epsilon = 0.03

def calc(roll, pitch, theta_3, theta_4, v, v_dot, omega, omega_dot, a=a, b=b, c=c, r_c=r_c, r_dw=r_dw, m=m, n=n, epsilon=epsilon, mu_rc=mu_rc, mu_rd=mu_rd, mu_tau=mu_tau,
         O_x=O_x, o_y = O_y, O_z = O_z):
    theta_pitch = pitch
    theta_roll = roll
    I = m*(4*c**2 + b**2)/12
    g = 9.8
"""
for s in get_all():
    funcstr += "    " + translate_str(s) + '\n'
    
funcstr += """
    d = {'N1': N_1, 'N2': N_2, 'N3': N_3, 'N4': N_4, 'sf3': s_f3, 'sf4': s_f4, 'st3': s_t3, 'st4': s_t4, 'R3x': R_3x, 'R3y': R_3y,
         'R4x': R_4x, 'R4y': R_4y, 'a1': alpha_1, 'a2': alpha_2, 'torque_1': torque_1, 'torque_2': torque_2}
    return d"""

if (False) :
    print(funcstr)
else:
    f = open('WheelchairFunc.py', 'w+')
    f.write(funcstr)
    f.close()